In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots

In [ ]:
from topo import SQuad
from poly import lagrange_list
from basis import LagrangeBasisQuad

In [ ]:
order   = 2
L       = 2*np.pi
n_elems = 2

x_max = L
y_max = L/2.0
x_vals = np.linspace(0, x_max, n_elems+1)
y_vals = np.linspace(0, y_max, n_elems+1)

vertices = np.zeros(((n_elems+1)**2, 2), dtype=np.double)
elem_to_vertex = np.zeros((n_elems**2, 4), dtype=np.int)

for i in range(n_elems):
    for j in range(n_elems):
        elem = i*n_elems+j
        elem_to_vertex[elem,0] = i*(n_elems+1)+j
        elem_to_vertex[elem,1] = i*(n_elems+1)+j+1
        elem_to_vertex[elem,2] = (i+1)*(n_elems+1)+j+1
        elem_to_vertex[elem,3] = (i+1)*(n_elems+1)+j

for i in range(n_elems+1):
    for j in range(n_elems+1):
        v = i*(n_elems+1)+j
        vertices[v,0] = x_vals[j]
        vertices[v,1] = y_vals[i]


In [ ]:
topo  = SQuad()
nodes = vertices[elem_to_vertex]
jacb  = topo.calc_jacb(nodes)
jacb_inv = topo.calc_jacb_inv(jacb)

### Basis Test Cases

* One at node, zero at all other nodes

In [ ]:
order = 3
basis = LagrangeBasisQuad(SQuad, order)

In [ ]:
basis.vertex_to_dof

In [ ]:
basis.edge_to_dof

In [ ]:
basis.bubble_to_dof

In [ ]:
x_vals = np.linspace(-1,1,order+1)
y_vals = x_vals
X, Y = np.meshgrid(x_vals, y_vals)
bp = basis.basis_polys[0]
k = 13
p = bp[k]
#plt.contour(X, Y, p(X, Y))

In [ ]:
p(X, Y).ravel()

In [ ]:
ref = np.array([X.ravel(), Y.ravel()]).T

coeffs = np.zeros(basis.n_dofs)
coeffs[k] = 1.0

np.all(np.max(np.abs(basis.eval_ref(coeffs, ref)-p(X,Y).ravel())<1e-12))